In [14]:
%%capture

!pip install cellpose seaborn scikit-image ipywidgets ipyfilechooser torch

# Create the 'images' folder if it doesn't exist
os.makedirs('images', exist_ok=True)

In [15]:
import os
import numpy as np
import pandas as pd
from skimage import io, measure, transform
from cellpose import models
import matplotlib.pyplot as plt
import scipy.stats as stats
from ipywidgets import Button, IntText, FloatProgress, Output, VBox, Layout
from ipyfilechooser import FileChooser
from IPython.display import clear_output
import seaborn as sns
from google.colab import files
import ipywidgets as widgets

In [17]:
upload_button = widgets.FileUpload(
    accept='image/*',  # Accept only image files
    multiple=True,  # Allow multiple file uploads
    description = 'Upload your images',
    layout=widgets.Layout(width='40%')
)
display(upload_button)

def handle_upload(change):
    for uploaded_filename in upload_button.value:
        content = upload_button.value[uploaded_filename]['content']
        with open(f'images/{uploaded_filename}', 'wb') as f:
            f.write(content)
    upload_button.value.clear()  # Clear the uploaded files
    print("Files uploaded successfully!")

upload_button.observe(handle_upload, names='value')

FileUpload(value={}, accept='image/*', description='Upload your images', layout=Layout(width='40%'), multiple=…

Files uploaded successfully!


In [19]:

# Initialize the Cellpose model for cell segmentation
model = models.Cellpose(model_type="cyto3")
# Create the 'images' folder if it doesn't exist
os.makedirs('images', exist_ok=True)

def create_mask_overlay(image, mask):
    """
    Create an overlay of the original image and the segmentation mask.

    Args:
    image (numpy.ndarray): The original image
    mask (numpy.ndarray): The segmentation mask

    Returns:
    matplotlib.figure.Figure: Figure object containing the overlay
    """
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.imshow(image, cmap="gray")  # Display original image in grayscale
    ax.imshow(mask, alpha=0.5, cmap="hsv")  # Overlay mask with 50% transparency
    ax.axis("off")  # Remove axes for cleaner visualization
    return fig


def process_images(folder_path, progress_bar, output_area):
    """
    Process all images in the specified folder using Cellpose segmentation.

    Args:
    folder_path (str): Path to the folder containing images
    progress_bar (ipywidgets.FloatProgress): Widget to show processing progress
    output_area (ipywidgets.Output): Widget to display processing status and results

    Returns:
    pandas.DataFrame: Filtered dataframe containing segmentation results
    """
    results = []
    overlay_dir = os.path.join(folder_path, "mask_overlays")
    os.makedirs(overlay_dir, exist_ok=True)

    # Get list of image files in the folder
    image_files = [
        f
        for f in os.listdir(folder_path)
        if f.lower().endswith((".tif", ".tiff", ".png"))
    ]
    total_images = len(image_files)

    for i, filename in enumerate(image_files):
        img_path = os.path.join(folder_path, filename)
        img = io.imread(img_path)
        # Resize image to 50% for faster processing
        img = transform.resize(img, (int(img.shape[0] * 0.5), int(img.shape[1] * 0.5)))

        try:
            # Perform Cellpose segmentation
            masks, flow, styles = models.CellposeModel(model_type="cyto3").eval(
                img, diameter=(diameter.value / 2), channels=[0, 0], flow_threshold=0.4
            )
        except ValueError as e:
            with output_area:
                print(f"Error processing {filename}: {str(e)}")
            continue

        # Update progress bar and output status
        progress_bar.value = (i + 1) / total_images
        with output_area:
            output_area.clear_output(wait=True)
            print(f"Processed image {i+1} of {total_images}: {filename}")

        # Create and save mask overlay
        fig = create_mask_overlay(img, masks)
        overlay_path = os.path.join(
            overlay_dir, f"{os.path.splitext(filename)[0]}_overlay.png"
        )
        fig.savefig(overlay_path, dpi=300, bbox_inches="tight")
        with output_area:
            display(fig)
        plt.close(fig)

        # Analyze properties of segmented regions
        props = measure.regionprops(masks)
        for j, prop in enumerate(props):
            # Calculate diameters
            # Multiply by 2 to account for earlier 50% image resize
            diameter_pixels = prop.equivalent_diameter * 2
            diameter_um = diameter_pixels * scale.value * 2

            # Append region properties to results
            results.append(
                {
                    "filename": filename,
                    "roi_index": j,
                    "diameter_pixels": prop.equivalent_diameter,
                    "diameter_um": diameter_um,
                    "area": prop.area,
                    "major_axis_length": prop.major_axis_length,
                    "minor_axis_length": prop.minor_axis_length,
                    "aspect_ratio": (
                        prop.major_axis_length / prop.minor_axis_length
                        if prop.minor_axis_length != 0
                        else np.nan
                    ),
                }
            )

    # Create dataframe and filter results
    df = pd.DataFrame(results)
    filtered_df = df[(df["aspect_ratio"] >= 0.8) & (df["aspect_ratio"] <= 1.2)]

    # Create and save histogram
    histogram_path = os.path.join(
        folder_path, "mask_overlays", "diameter_histogram.png"
    )
    histogram_fig = create_histogram(filtered_df["diameter_um"].values, histogram_path)

    # Display histogram inline after overlay masks
    with output_area:
        display(histogram_fig)
    plt.close(histogram_fig)

    return filtered_df


# Set up GUI components
diameter = IntText(
    value=100,
    description="Estimated diameter (pixels):",
    style={"description_width": "initial"},
    layout=Layout(width="300px"),
)
scale = IntText(
    value=1,
    description="Scale (um/px):",
    style={"description_width": "initial"},
    layout=Layout(width="150px"),
)
process_button = Button(description="Process Images")
progress_bar = FloatProgress(min=0, max=1, description="Progress:", bar_style="info")
output_area = Output()


def create_histogram(diameters, output_path):
    """
    Create a histogram of diameters with a vertical line at the average.

    Args:
    diameters (numpy.ndarray): Array of diameter values
    output_path (str): Path to save the histogram image

    Returns:
    matplotlib.figure.Figure: Figure object containing the histogram
    """
    plt.figure(figsize=(10, 6))

    # Create histogram
    # n, bins, patches = ax.hist(diameters, bins=80, edgecolor='black')
    ax = sns.histplot(
        diameters,
        bins=120,
        kde=True,
        kde_kws={"bw_adjust": 0.3},
        line_kws={"linewidth": 2, "linestyle": "-", "color": "blue"},
    )
    # Calculate and plot average line
    average = np.mean(diameters)
    stdev = np.std(diameters)
    # plot the average line
    ax.axvline(average, color="r", linestyle="dashed", linewidth=2)
    # calculate and plot the standard deviation lines
    ax.axvline(average + stdev, color="b", linestyle="dashed", linewidth=2)
    ax.axvline(average - stdev, color="b", linestyle="dashed", linewidth=2)

    # Add labels and title
    ax.set_xlabel("Diameter (µm)")
    ax.set_ylabel("Frequency")
    ax.set_title("Histogram of Cell Diameters")

    # Add text for average

    average_label = f"Average: {average:.2f} ± {stdev:.2f} µm"

    ax.text(
        average * 1.1,
        ax.get_ylim()[1] * 0.9,
        average_label,
        verticalalignment="top",
        horizontalalignment="left",
        color="b",
        fontweight="bold",
    )

    # Save the figure
    plt.savefig(output_path, dpi=300, bbox_inches="tight")
    return ax.get_figure()


def on_button_clicked(b):
    """
    Event handler for the "Process Images" button click.
    Initiates image processing and saves results.

    Args:
    b (ipywidgets.Button): The button widget (not used in the function)
    """
    clear_output(wait=True)
    folder_path = "/content/images"
    # Count the number of image files in the selected folder
    image_files = [
        f
        for f in os.listdir(folder_path)
        if f.lower().endswith((".tif", ".tiff", ".png"))
    ]
    total_images = len(image_files)

    with output_area:
        print(f"Processing {total_images} image(s) in {folder_path}...")

    # Process images and get results
    results_df = process_images(folder_path, progress_bar, output_area)

    if results_df.empty:
        print(
            "No results to export. Please check if there were any errors during processing and if you have images on the selected folder."
        )
        return

    # Save results to Excel file
    output_path = os.path.join(folder_path, "segmentation_results.xlsx")
    results_df.to_excel(output_path, index=False)
    with output_area:
        print(f"Results exported to {output_path}")
        print(f"Mask overlays saved in {os.path.join(folder_path, 'mask_overlays')}")
        histogram_path = os.path.join(
            folder_path, "mask_overlays", "diameter_histogram.png"
        )
        print(f"Diameter histogram saved as {histogram_path}")


# Attach the click event handler to the process button
process_button.on_click(on_button_clicked)

# Display GUI components
display(
    VBox([diameter, scale, process_button, progress_bar, output_area])
)